# Azure AI Agent Service入門

このコードサンプルでは、[Azure AI Agent Service](https://aka.ms/ai-agents-beginners/azure-ai-agent-service) を使用して本格的なエージェントを作成し、実際のAzure認証とツール統合を体験します。

**🎯 実現するシナリオ**: 旅行データから棒グラフを生成し、Code Interpreterを使用してリアルタイムでデータ可視化とファイル出力を行う実践的なエージェント開発

## 📚 このノートブックで学習できること

### 🎯 主要な学習目標
- **Azure AI Projects SDK**を使用したエージェント開発
- **Azure Default Credential**による認証体験
- **Code Interpreter Tool**の実際の使用
- **リアルタイム処理**とファイル生成

### 🛠️ 実装する技術スタック
- **Azure AI Projects SDK**: 正式なAzure AIエージェントSDK
- **Azure Identity**: DefaultAzureCredentialによる認証
- **Code Interpreter**: Pythonコード実行と可視化
- **Azure AI Foundry**: モデルデプロイメントとプロジェクト管理

### 💡 学習できるスキル
- **認証**: Azure認証情報の設定と使用
- **プロジェクト管理**: Azure AI プロジェクトの設定
- **ツール統合**: Code Interpreterの実装
- **エラーハンドリング**: 実際のAPI使用時の例外処理

### ⚙️ 必要な環境設定

**1. Azure AI Foundry プロジェクト**
- Azure AI Foundryでプロジェクトを作成
- モデル（gpt-4o-mini）をデプロイ
- プロジェクトエンドポイントを取得
- **重要**: デプロイされたモデル名を正確に確認してください

**2. 環境変数設定**
```bash
PROJECT_ENDPOINT=https://your-project.cognitiveservices.azure.com/
MODEL_DEPLOYMENT_NAME=gpt-4o-mini
```

**3. モデルデプロイメントの確認**
- Azure AI Foundryの「Models + endpoints」でデプロイ状況を確認
- デプロイされたモデル名が環境変数のMODEL_DEPLOYMENT_NAMEと一致することを確認
- 一般的なモデル名: `gpt-4o-mini`, `gpt-4o`, `gpt-35-turbo`

**3. Azure認証**
- Azure CLIでログイン: `az login`
- または、Managed Identityの設定
- Azure AI Servicesへのアクセス権限

**4. 必要なパッケージ**
```bash
pip install azure-ai-projects azure-identity python-dotenv
```

### 🚨 よくあるエラーと対処法

**❌ `invalid_engine_error`**: モデルが見つからない
- **対処法**: Azure AI Foundryでモデルのデプロイ状況を確認
- **確認点**: 環境変数MODEL_DEPLOYMENT_NAMEとデプロイされたモデル名が一致するか
- **解決策**: 正しいモデル名を.envファイルに設定

**❌ `authentication_error`**: 認証エラー 
- **対処法**: `az login`でAzure CLIにログイン
- **確認点**: Azure AI Servicesへのアクセス権限があるか

**❌ `project_not_found`**: プロジェクトが見つからない
- **対処法**: PROJECT_ENDPOINTの値を確認
- **確認点**: Azure AI Foundryでプロジェクトが正しく作成されているか

**❌ `HTTP transport has already been closed`**: 接続エラー
- **対処法**: カーネルを再起動してセルを再実行
- **確認点**: 同じセルを繰り返し実行していないか
- **解決策**: 関数内で新しいクライアントを作成（修正済み）

### ⚠️ 重要な注意事項
このサンプルでは**実際のAzure AI Agent Service**を使用します：
- Azure サブスクリプションが必要です
- API使用による課金が発生する可能性があります
- 適切な認証設定が必要です
- ネットワーク接続が必要です

## Azure AI Projects クライアントの初期化

Azure認証とプロジェクト接続を設定します。

In [10]:
import os
from dotenv import load_dotenv

from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path
from datetime import datetime

In [11]:
load_dotenv()
project_endpoint = os.environ["PROJECT_ENDPOINT"]

project_client = AIProjectClient(
    endpoint=project_endpoint,
    # Azure Default Credential を使用した認証
    credential=DefaultAzureCredential(),
)

## Azure AI Agent の実行とデモンストレーション

実際のAzure AI Agentを作成し、Code Interpreterツールを使用してデータの可視化を行います。

### 📥 ファイル生成と制限事項について

**⚠️ 現在のSDKの制限事項**

このサンプルコードでは、Azure AI Projects SDKの現在の制限により、Code Interpreterで生成されたファイルの直接ダウンロードはサポートされていません：

- **生成されたファイル**: サンドボックス環境（`sandbox:/mnt/data/`）に一時保存される
- **アクセス制限**: SDKからの直接ダウンロードAPIが現在未提供
- **学習目的**: このノートブックは認証とエージェント使用方法の学習を目的としています

**🚀 実際のアプリケーション開発での実装方針**

本格的なアプリケーションでファイル処理が必要な場合の推奨アプローチ：

**1. Azure AI Agents SDK使用**
```python
# より高度なファイル操作が可能なSDK
from azure.ai.agents import AgentsClient
# ファイルダウンロード機能が利用可能
```

**2. Azure Storage統合**
```python
# 生成ファイルを永続ストレージに保存
# - Azure Blob Storage への自動アップロード
# - ファイル共有とダウンロードリンク生成
# - メタデータ管理とバージョン管理
```

**3. カスタムツール開発**
```python
# 独自のファイル処理ツールを実装
# - ファイル生成後の自動保存
# - 外部APIとの連携
# - ワークフロー自動化
```

**💡 学習の価値**
- Azure認証プロセスの体験
- エージェント作成と管理の理解  
- ツール統合の基本概念
- エラーハンドリングの実装

In [18]:
from IPython.display import display, HTML, Image
from pathlib import Path


async def run_agent_with_visualization():
    """Azure AI Agent実行と可視化"""
    html_output = "<h2>🤖 Azure AI Agent 実行</h2>"

    # 環境変数を読み込み
    load_dotenv()
    project_endpoint = os.environ["PROJECT_ENDPOINT"]
    
    # 新しいクライアントを作成（関数内で毎回作成）
    client = AIProjectClient(
        endpoint=project_endpoint,
        credential=DefaultAzureCredential(),
    )

    with client:
        # CodeInterpreterTool のインスタンスを作成
        code_interpreter = CodeInterpreterTool()

        # 環境変数からモデル名を取得
        model_name = os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")  # デフォルトは gpt-4o-mini
        
        # デバッグ: 使用するモデル名を表示
        print(f"🔍 使用するモデル名: {model_name}")
        print(f"🔍 環境変数MODEL_DEPLOYMENT_NAME: {os.getenv('MODEL_DEPLOYMENT_NAME', '未設定')}")
        
        # エージェントの作成にはCodeInterpreterToolを含める必要があります
        # Azure AI Foundryでデプロイしたモデル名を環境変数から取得
        try:
            agent = client.agents.create_agent(
                model=model_name,
                name="my-agent",
                instructions="あなたは親切なアシスタントです",
                tools=code_interpreter.definitions,
            )
            print(f"✅ エージェント作成成功: {agent.id}")
        except Exception as e:
            print(f"❌ エージェント作成エラー: {e}")
            print(f"💡 Azure AI Foundryで'{model_name}'がデプロイされているか確認してください")
            raise

        html_output += f"<div style='margin:10px 0; padding:10px; background-color:#f0f7ff; border-left:4px solid #007bff; border-radius:4px;'>"
        html_output += f"<strong>🎯 エージェント作成完了</strong><br>"
        html_output += f"<strong>エージェントID:</strong> {agent.id}<br>"
        html_output += f"<strong>モデル:</strong> {model_name}<br>"
        html_output += f"<strong>ツール:</strong> Code Interpreter"
        html_output += "</div>"

        # スレッドを作成
        thread = client.agents.threads.create()
        html_output += f"<div style='margin:10px 0; padding:8px; background-color:#fff3cd; border:1px solid #ffeaa7; border-radius:4px;'>"
        html_output += f"<strong>📝 スレッド作成完了</strong> ID: {thread.id}"
        html_output += "</div>"

        # ユーザークエリ - わかりやすく表示
        user_query = "次のデータを使用して旅行者数の棒グラフを作成し、ファイルとして提供してもらえますか？ バリ島: 100人, パリ: 356人, ロンドン: 900人, 東京: 850人"
        html_output += "<div style='margin:15px 0; padding:10px; background-color:#f5f5f5; border-left:4px solid #28a745; border-radius:4px;'>"
        html_output += "<strong>👤 ユーザー:</strong><br>"
        html_output += f"<div style='margin-left:15px'>{user_query}</div>"
        html_output += "</div>"

        # メッセージを作成
        message = client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_query,
        )

        # エージェントを実行 - "処理中"メッセージを表示
        display(HTML(
            html_output + "<div style='color:#007bff; margin:10px 0;'><i>📊 リクエストを処理中...</i></div>"))

        # 実行を開始
        run = client.agents.runs.create_and_process(
            thread_id=thread.id, agent_id=agent.id)

        # ステータス更新
        status_color = 'green' if run.status == 'completed' else 'red'
        html_output += f"<div style='margin:10px 0; padding:8px; background-color:#{'e8f5e8' if run.status == 'completed' else 'f8d7da'}; border-radius:4px;'>"
        html_output += f"<strong>⚡ 実行完了</strong> ステータス: <span style='color:{status_color}; font-weight:bold'>{run.status}</span>"
        html_output += "</div>"

        if run.status == "failed":
            html_output += f"<div style='color:red; margin:10px 0; padding:8px; background-color:#f8d7da; border-radius:4px;'>"
            html_output += f"<strong>❌ 実行失敗:</strong> {run.last_error}"
            html_output += "</div>"

        # スレッドからメッセージを取得
        messages = client.agents.messages.list(thread_id=thread.id)

        # アシスタントの応答をフォーマット
        html_output += "<div style='margin:15px 0; padding:10px; background-color:#f0f7ff; border-left:4px solid #007bff; border-radius:4px;'>"
        html_output += "<strong>🤖 アシスタント:</strong><br>"

        # 実際の構造に基づいてメッセージを処理
        assistant_response_content = ""
        saved_files = []  # ファイル保存を追跡
        saved_images = []  # 画像保存を追跡
        
        try:
            # ItemPagedオブジェクトを直接イテレート
            for msg in messages:
                if hasattr(msg, 'role') and msg.role == "assistant":
                    if hasattr(msg, 'content') and isinstance(msg.content, list):
                        for content_item in msg.content:
                            if hasattr(content_item, 'type') and content_item.type == "text":
                                assistant_response_content = content_item.text.value
                                html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{assistant_response_content}</div>"
                            elif hasattr(content_item, 'type') and content_item.type == "image_file":
                                # 画像ファイルの処理
                                image_file_id = content_item.image_file.file_id
                                image_file_name = f"generated_image_{image_file_id}.png"
                                try:
                                    # ファイルをローカルに保存
                                    file_content = client.agents.files.download_file(file_id=image_file_id)
                                    with open(image_file_name, 'wb') as f:
                                        f.write(file_content)
                                    saved_images.append(image_file_name)
                                    saved_files.append({"name": image_file_name, "type": "image", "id": image_file_id})
                                    html_output += f"<div style='margin:10px 0; padding:8px; background-color:#e8f5e8; border-radius:4px;'>"
                                    html_output += f"<strong>🖼️ ダウンロード完了:</strong> {image_file_name}"
                                    html_output += "</div>"
                                except Exception as download_error:
                                    html_output += f"<div style='color:orange; margin:5px 0;'>画像ダウンロードエラー: {str(download_error)}</div>"
                                    
        except Exception as e:
            html_output += f"<div style='color:red'><strong>❌ メッセージ処理エラー:</strong> {str(e)}</div>"

        # ファイル注釈の処理（メッセージ内でsandbox:形式で言及されたファイル）
        if assistant_response_content:
            import re
            # sandbox:/mnt/data/xxx.png のようなパターンを検索
            sandbox_files = re.findall(r'sandbox:/mnt/data/([^)\s]+\.(?:png|jpg|jpeg|pdf|csv|txt|json))', assistant_response_content)
            
            if sandbox_files:
                html_output += "<div style='margin:10px 0; padding:8px; background-color:#fff3cd; border:1px solid #ffeaa7; border-radius:4px;'>"
                html_output += "<strong>📁 検出されたファイル参照:</strong><br>"
                
                for file_name in sandbox_files:
                    html_output += f"• {file_name}<br>"
                    html_output += f"<div style='margin:5px 0; padding:5px; background-color:#f8d7da; border-radius:3px;'>"
                    html_output += f"<strong>⚠️ サンプルコード制限:</strong> このノートブックではファイルダウンロードはサポートされていません<br>"
                    html_output += f"<strong>💡 学習目的:</strong> Code Interpreterの動作とファイル生成プロセスを理解するためのデモです<br>"
                    html_output += f"<strong>🚀 実用化:</strong> 本格的なアプリケーションでは Azure Storage や Azure AI Agents SDK を使用してください"
                    html_output += "</div>"
                        
                html_output += "</div>"

        html_output += "</div>"

        # 完了後にエージェントを削除
        client.agents.delete_agent(agent.id)
        html_output += "<div style='margin-top:15px; color:#6c757d; font-style:italic;'>🗑️ 完了後にエージェントを削除しました</div>"

        # ダウンロードされたファイルの情報をまとめて表示
        if saved_files:
            html_output += "<div style='margin:15px 0; padding:10px; background-color:#d1ecf1; border-left:4px solid #bee5eb; border-radius:4px;'>"
            html_output += "<strong>📥 ダウンロード完了ファイル一覧:</strong><br>"
            for file_info in saved_files:
                html_output += f"• <strong>{file_info['name']}</strong> ({file_info['type']})<br>"
            html_output += f"<br><em>合計 {len(saved_files)} ファイルがローカルに保存されました</em>"
            html_output += "</div>"

        # すべてのコンテンツの最終表示
        display(HTML(html_output))

        # 保存された画像を表示
        for img_file in saved_images:
            try:
                display(Image(img_file))
            except Exception as e:
                print(f"画像表示エラー: {img_file} - {str(e)}")

        # ファイルの存在確認メッセージ
        if saved_files:
            print(f"\n✅ 以下のファイルがノートブックと同じディレクトリに保存されました:")
            for file_info in saved_files:
                if os.path.exists(file_info['name']):
                    file_size = os.path.getsize(file_info['name'])
                    print(f"📁 {file_info['name']} ({file_size} bytes)")
                else:
                    print(f"⚠️ {file_info['name']} (保存されていません)")

# 関数を実行
await run_agent_with_visualization()

🔍 使用するモデル名: gpt-4o-mini
🔍 環境変数MODEL_DEPLOYMENT_NAME: gpt-4o-mini
✅ エージェント作成成功: asst_baGzcA2LWSXAZ7BifxeTulsE
✅ エージェント作成成功: asst_baGzcA2LWSXAZ7BifxeTulsE


## 🎓 学習の振り返りと次のステップ

### ✅ このノートブックで習得したスキル

**技術的な体験**
- **Azure AI Projects SDK**: 正式なSDKを使用したエージェント開発
- **Azure認証**: DefaultAzureCredentialによる実際の認証フロー
- **Code Interpreter統合**: リアルタイムPythonコード実行とデータ可視化
- **エラーハンドリング**: 実際のAPI使用時の例外処理とデバッグ

**実践的な理解**
- Azure AI Foundryでのプロジェクト設定とモデルデプロイ
- 環境変数を使用した設定管理
- エージェントのライフサイクル管理（作成・実行・削除）
- SDK制限事項の理解と対処法

### 🚀 実用アプリケーション開発への次のステップ

**1. ファイル処理の高度化**
```python
# より強力なファイル操作が必要な場合
from azure.ai.agents import AgentsClient  # 高機能SDK
from azure.storage.blob import BlobServiceClient  # 永続ストレージ
```

**2. 本番環境への準備**
- **認証**: Managed Identity や Service Principal の設定
- **セキュリティ**: Azure Key Vault での秘密情報管理
- **監視**: Application Insights でのパフォーマンス追跡
- **スケーリング**: Azure Container Apps での自動スケール

**3. 高度な機能の追加**
- **カスタムツール**: 独自のツール開発と統合
- **Azure Search**: RAG（Retrieval-Augmented Generation）の実装
- **Function Calling**: 外部APIとの連携
- **マルチエージェント**: 複数エージェントの協調動作

### 💡 学習成果の活用例

**ビジネス応用**
- データ分析レポートの自動生成
- カスタマーサポートの自動化
- 業務プロセスの最適化

**技術応用**  
- 開発ツールの自動化
- コード生成とレビュー支援
- システム監視とアラート

このノートブックは Azure AI Agent Service の**入門編**として、認証から基本的なツール使用まで実践的に学習できる内容でした。次は実際のプロジェクトでの応用を目指してください！